In [1]:
import numpy as np
from surrogate_model import Aleatoric_NN, Epistemic_NN
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import copy
import pickle

## Active Learning experiment

In [2]:
n_init = 2000
n_test = 2000
k = [500, 1000, 2000, 4000, 8000, 16000, 32000]
m=4
iterations = 10
active_learning_mses = []
baseline_mses = []
baseline_ns = []

In [3]:
dom = ([-3,3],[-2,2])

def six_hump_camel_function(X):
    x = X[:,0]
    y = X[:,1]

    x2 = np.power(x,2)
    x4 = np.power(x,4)
    y2 = np.power(y,2)

    return (4.0 - 2.1 * x2 + (x4 / 3.0)) * x2 + x*y + (-4.0 + 4.0 * y2) * y2

In [4]:
grid = 1200
X1 = np.linspace(dom[0][0], dom[0][1], grid)
X2 = np.linspace(dom[1][0], dom[1][1], grid)
x1, x2 = np.meshgrid(X1, X2)
X = np.hstack((x1.reshape(grid*grid,1),x2.reshape(grid*grid,1)))


In [5]:
rng = np.random.RandomState(42)
test_indices = np.random.randint(0, (grid*grid), size = n_test)
testset_X = X[test_indices,:]
X = np.delete(X, test_indices, axis=0)

In [6]:
print(testset_X.shape)
print(X.shape)

(2000, 2)
(1438005, 2)


In [7]:
testset_y = six_hump_camel_function(testset_X)

In [8]:
init_set = X[np.random.randint(0,X.shape[0],n_init),:]

In [9]:
testset_y.shape

(2000,)

In [10]:
y = six_hump_camel_function(init_set)

In [11]:
for j in k:
    total_n = 0
    nn = Epistemic_NN()
    start_set = copy.deepcopy(init_set)
    y_new = copy.deepcopy(y)
    nn.create_model(start_set)
    nn.fit(start_set, y_new, epochs = 25)
    for i in range(iterations):
        new_set = X[np.random.randint(0,X.shape[0],m*j),:]
        evaluation_m, evaluation_s = nn.predict(new_set)
        #print(evaluation_s)
        highest_error_indices = np.argpartition(evaluation_s, -j)[-j:]
        y_new_ = six_hump_camel_function(new_set[highest_error_indices,:])
        print(f"Mean prediction uncertainty: {np.mean(evaluation_s[highest_error_indices])}")
        #start_set = np.concatenate((start_set,  new_set[highest_error_indices]), axis = 0)
        print(f"trainset size: {start_set.shape[0]}")
        #y_new = np.concatenate((y_new, y_new_), axis=0)
        nn.updateModel(new_set[highest_error_indices,:], y_new_, epochs = 20, verbose=0)
        total_n+=j
        print(f"finished n={j}, iteration={i}")
    nn.model.save(f"AL_{j}.h5")
        
    test_m, test_s = nn.predict(testset_X)
    active_learning_mses.append(mean_squared_error(testset_y, test_m))
    print(f"mse: {active_learning_mses}")
    baseline_ns.append(total_n)
    print(f"baseline_ns: {baseline_ns}")

Mean prediction uncertainty: 24.362551271765135
trainset size: 2000
finished n=500, iteration=0
Mean prediction uncertainty: 21.752165954428797
trainset size: 2000
finished n=500, iteration=1
Mean prediction uncertainty: 20.23165186434713
trainset size: 2000
finished n=500, iteration=2
Mean prediction uncertainty: 18.941023048514936
trainset size: 2000
finished n=500, iteration=3
Mean prediction uncertainty: 18.077096483971427
trainset size: 2000
finished n=500, iteration=4
Mean prediction uncertainty: 17.765472253291627
trainset size: 2000
finished n=500, iteration=5
Mean prediction uncertainty: 14.621235226585949
trainset size: 2000
finished n=500, iteration=6
Mean prediction uncertainty: 14.84754636207454
trainset size: 2000
finished n=500, iteration=7
Mean prediction uncertainty: 14.687340388097699
trainset size: 2000
finished n=500, iteration=8
Mean prediction uncertainty: 14.217654548353003
trainset size: 2000
finished n=500, iteration=9
mse: [44.45543047497739]
baseline_ns: [500